In [ ]:
!pip install torchsummaryX --quiet

In [ ]:
!nvidia-smi

Fri Dec  9 18:09:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    28W /  70W |   3710MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
#from torchsummaryX import summary
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import os
import torchvision
from matplotlib import image,pyplot

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip -qo '/content/drive/MyDrive/IDL Project/sonar_camera.zip'

In [ ]:
os.listdir("sonar_camera_group/camera_data")
!rmdir "sonar_camera_group/camera_data/.ipynb_checkpoints"

rmdir: failed to remove 'sonar_camera_group/camera_data/.ipynb_checkpoints': No such file or directory


In [ ]:
os.listdir("sonar_camera_group/sonar_data")
!rmdir "sonar_camera_group/sonar_data/.ipynb_checkpoints"

rmdir: failed to remove 'sonar_camera_group/sonar_data/.ipynb_checkpoints': No such file or directory


In [ ]:
from skimage.util import random_noise
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Custom Transform for adding noise
class Add_Noise(object):

    def __init__(self,mode):
      assert isinstance(mode,str)
      self.mode=mode

    def __call__(self, image):
      image = torch.tensor(random_noise(image, mode=self.mode, mean=0, var=0.1, clip=True),dtype=torch.float32)   
      #print(image)
      return image

In [ ]:
 # TRAIN TEST SPLIT

import torch
import torchvision
transforms_camera=torchvision.transforms.Compose([torchvision.transforms.ToTensor(),Add_Noise("gaussian")])
transforms_sonar=torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

camera_dataset = torchvision.datasets.ImageFolder(root='sonar_camera_group/camera_data',transform=transforms_camera)
sonar_dataset = torchvision.datasets.ImageFolder(root='sonar_camera_group/sonar_data', transform=transforms_sonar)

train_size = int(0.8 * len(camera_dataset))
val_size = len(camera_dataset) - train_size

camera_train, camera_test = torch.utils.data.random_split(camera_dataset, [train_size, val_size], generator=torch.Generator().manual_seed(0))
sonar_train, sonar_test = torch.utils.data.random_split(sonar_dataset, [train_size, val_size], generator=torch.Generator().manual_seed(0))

# check if same labels
#for id in range(20):
    #print(camera_train[id])
    #print(sonar_train[id])
    #print('\n')


In [ ]:

train_loader_sonar = torch.utils.data.DataLoader(sonar_train, batch_size = 8, 
                                           shuffle = False,num_workers = 2, pin_memory = True)
val_loader_sonar = torch.utils.data.DataLoader(sonar_test, batch_size = 8, 
                                         shuffle = False, num_workers = 2)
train_loader_camera = torch.utils.data.DataLoader(camera_train, batch_size = 8, 
                                           shuffle = False,num_workers = 2, pin_memory = True)
val_loader_camera = torch.utils.data.DataLoader(camera_test, batch_size = 8, 
                                         shuffle = False, num_workers = 2)

In [ ]:
#for i,data in train_loader:
  #print(i.shape)
  #break
print(len(sonar_train),len(sonar_test))
print(len(camera_train),len(camera_test))

1096 275
1096 275


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
class ImageClassifier(torch.nn.Module):

    def __init__(self, num_classes=5):
        super().__init__()
        
        self.net=nn.Sequential(nn.Conv2d(3,32,3,2),
                               nn.BatchNorm2d(32),
                               nn.ReLU(),
                               nn.Dropout(p=0.2),
                               nn.Conv2d(32,64,2,2),
                               nn.BatchNorm2d(64),
                               nn.ReLU(),
                               nn.Dropout(p=0.2),
                               nn.Conv2d(64,64,2,2),
                               nn.BatchNorm2d(64),
                               nn.ReLU(),
                               nn.Dropout(p=0.2),
                               nn.Conv2d(64,64,2,2),
                               nn.BatchNorm2d(64),
                               nn.ReLU(),
                               torch.nn.Flatten(),
                               nn.Linear(61504,512))

        self.cls_layer = torch.nn.Linear(512,num_classes)#TODO
        
        
    
    def forward(self, x, return_feats=False):

        feats=self.net(x)
        out=self.cls_layer(feats)
        return out
        
            
camera_model = ImageClassifier().to(device)
sonar_model=ImageClassifier().to(device)
summary(camera_model,(3,512,512))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 255, 255]             896
       BatchNorm2d-2         [-1, 32, 255, 255]              64
              ReLU-3         [-1, 32, 255, 255]               0
           Dropout-4         [-1, 32, 255, 255]               0
            Conv2d-5         [-1, 64, 127, 127]           8,256
       BatchNorm2d-6         [-1, 64, 127, 127]             128
              ReLU-7         [-1, 64, 127, 127]               0
           Dropout-8         [-1, 64, 127, 127]               0
            Conv2d-9           [-1, 64, 63, 63]          16,448
      BatchNorm2d-10           [-1, 64, 63, 63]             128
             ReLU-11           [-1, 64, 63, 63]               0
          Dropout-12           [-1, 64, 63, 63]               0
           Conv2d-13           [-1, 64, 31, 31]          16,448
      BatchNorm2d-14           [-1, 64,

In [ ]:
class Network_fused(torch.nn.Module):

    def __init__(self, num_classes=5):
        super().__init__()
        
        self.net=nn.Sequential(nn.Conv2d(6,36,3,2,groups=6),
                               nn.BatchNorm2d(36),
                               nn.ReLU(),
                               nn.Dropout(p=0.2),
                               nn.Conv2d(36,72,2,2,groups=36),
                               nn.BatchNorm2d(72),
                               nn.ReLU(),
                               nn.Dropout(p=0.2),
                               nn.Conv2d(72,72,2,2,groups=72),
                               nn.BatchNorm2d(72),
                               nn.ReLU(),
                               nn.Dropout(p=0.2),
                               nn.Conv2d(72,72,2,2,groups=72),
                               nn.BatchNorm2d(72),
                               nn.ReLU(),
                               torch.nn.Flatten())

        self.cls_layer = torch.nn.Linear(69192,num_classes)#TODO
        
        
    
    def forward(self, x, return_feats=False):

        
        feats=self.net(x)
        out=self.cls_layer(feats)
        return out
        
            
model2 = Network_fused().to(device)
summary(model2,(6,512,512))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 36, 255, 255]             360
       BatchNorm2d-2         [-1, 36, 255, 255]              72
              ReLU-3         [-1, 36, 255, 255]               0
           Dropout-4         [-1, 36, 255, 255]               0
            Conv2d-5         [-1, 72, 127, 127]             360
       BatchNorm2d-6         [-1, 72, 127, 127]             144
              ReLU-7         [-1, 72, 127, 127]               0
           Dropout-8         [-1, 72, 127, 127]               0
            Conv2d-9           [-1, 72, 63, 63]             360
      BatchNorm2d-10           [-1, 72, 63, 63]             144
             ReLU-11           [-1, 72, 63, 63]               0
          Dropout-12           [-1, 72, 63, 63]               0
           Conv2d-13           [-1, 72, 31, 31]             360
      BatchNorm2d-14           [-1, 72,

In [ ]:
def train(model, camera_dataloader,sonar_dataloader, optimizer, criterion,mode="camera"):
    
    model.train()

    # Progress Bar 
    batch_bar = tqdm(total=len(camera_dataloader), dynamic_ncols=True, leave=False, position=0, desc='Train', ncols=5) 
    
    num_correct = 0
    total_loss = 0
    if mode=="camera":
      dataloader=camera_dataloader
    elif mode=="sonar":
      dataloader=sonar_dataloader
    else:
      dataloader=zip(camera_dataloader,sonar_dataloader)

    for i, data in enumerate(dataloader):
        
        if mode=="camera" or mode=="sonar":
          images=data[0]
          labels=data[1]
        else:
          image_camera=data[0][0]
          labels=data[0][1] #Same labels for both 
          image_sonar=data[1][0]
          images=torch.concat([image_camera,image_sonar],dim=1)
        
        optimizer.zero_grad() # Zero gradients

        images, labels = images.to(device), labels.to(device)
        
        with torch.cuda.amp.autocast(): # This implements mixed precision. Thats it! 
            outputs = model(images)
            #print(outputs.shape)
            #print(labels.shape)
            loss = criterion(outputs, labels)

        # Update no. of correct predictions & loss as we iterate
        num_correct += int((torch.argmax(outputs, axis=1) == labels).sum())
        total_loss += float(loss.item())

        # tqdm lets you add some details so you can monitor training as you train.
        batch_bar.set_postfix(
            acc="{:.04f}%".format(100 * num_correct / (8*(i + 1))),
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            num_correct=num_correct,
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        
        loss.backward()
        optimizer.step()

        # TODO? Depending on your choice of scheduler,
        # You may want to call some schdulers inside the train function. What are these?
      
        batch_bar.update() # Update tqdm bar

    batch_bar.close() # You need this to close the tqdm bar

    acc = 100 * num_correct / (8* len(camera_dataloader))
    total_loss = float(total_loss / len(camera_dataloader))

    return acc, total_loss

In [ ]:
def validate(model, camera_dataloader, sonar_dataloader,criterion,mode="camera"):
  
    model.eval()
    batch_bar = tqdm(total=len(camera_dataloader), dynamic_ncols=True, position=0, leave=False, desc='Val', ncols=5)

    num_correct = 0.0
    total_loss = 0.0

    if mode=="camera":
      dataloader=camera_dataloader
    elif mode=="sonar":
      dataloader=sonar_dataloader
    else:
      dataloader=zip(camera_dataloader,sonar_dataloader)

    for i, data in enumerate(dataloader):
        
        if mode=="camera" or mode=="sonar":
          images=data[0]
          labels=data[1]
        else:
          image_camera=data[0][0]
          labels=data[0][1] #Same labels for both 
          image_sonar=data[1][0]
          images=torch.concat([image_camera,image_sonar],dim=1)


        # Move images to device
        images, labels = images.to(device), labels.to(device)
        
        # Get model outputs
        with torch.inference_mode():
            outputs = model(images)
            loss = criterion(outputs, labels)

        num_correct += int((torch.argmax(outputs, axis=1) == labels).sum())
        total_loss += float(loss.item())

        batch_bar.set_postfix(
            acc="{:.04f}%".format(100 * num_correct / (8*(i + 1))),
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            num_correct=num_correct)
        batch_bar.update()
        
    batch_bar.close()
    acc = 100 * num_correct / (8* len(camera_dataloader))
    total_loss = float(total_loss / len(sonar_dataloader))
    return acc, total_loss

In [ ]:
import gc
gc.collect() # These commands help you when you face CUDA OOM error
torch.cuda.empty_cache()

###UNIMODAL SONAR MODEL

In [ ]:
criterion = torch.nn.CrossEntropyLoss() # TODO: What loss do you need for a multi class classification problem?
optimizer = torch.optim.Adam(sonar_model.parameters(), lr=1e-3)
# scaler = torch.cuda.amp.GradScaler()

In [ ]:

for epoch in range(0,30):

    train_acc, train_loss = train(sonar_model, train_loader_camera,train_loader_sonar,optimizer, criterion,mode="sonar")
    
    print("\nEpoch {}/{}: \nTrain Acc {:.04f}%\t Train Loss {:.04f}\t".format(
        epoch + 1,
        30,
        train_acc,
        train_loss))
    
    val_acc, val_loss = validate(sonar_model, val_loader_camera,val_loader_sonar, criterion,mode="sonar")
    print("Val Acc {:.04f}%\t Val Loss {:.04f}".format(val_acc, val_loss))


Epoch 1/30: 
Train Acc 49.8175%	 Train Loss 5.4350	


Val Acc 64.2857%	 Val Loss 0.8738



Epoch 2/30: 
Train Acc 72.7190%	 Train Loss 0.6938	


Val Acc 71.0714%	 Val Loss 0.7355



Epoch 3/30: 
Train Acc 77.7372%	 Train Loss 0.5392	


...

Val Acc 63.2143%	 Val Loss 5.2713



Epoch 29/30: 
Train Acc 85.4927%	 Train Loss 0.5339	


Val Acc 68.9286%	 Val Loss 3.5440



Epoch 30/30: 
Train Acc 86.4964%	 Train Loss 0.4436	


Val Acc 71.7857%	 Val Loss 3.7904


In [ ]:
torch.save(sonar_model.state_dict(), "/content/drive/MyDrive/IDL Project/sonar_model_final1.pt")

###UNIMODAL CAMERA MODEL

In [ ]:
criterion = torch.nn.CrossEntropyLoss() # TODO: What loss do you need for a multi class classification problem?
optimizer = torch.optim.Adam(camera_model.parameters(), lr=1e-3)
# scaler = torch.cuda.amp.GradScaler()

In [ ]:
for epoch in range(0,30):

    train_acc, train_loss = train(camera_model, train_loader_camera,train_loader_sonar,optimizer, criterion,mode="camera")
    
    print("\nEpoch {}/{}: \nTrain Acc {:.04f}%\t Train Loss {:.04f}\t".format(
        epoch + 1,
        30,
        train_acc,
        train_loss))
    
    val_acc, val_loss = validate(camera_model, val_loader_camera,val_loader_sonar, criterion,mode="camera")
    print("Val Acc {:.04f}%\t Val Loss {:.04f}".format(val_acc, val_loss))


Epoch 1/30: 
Train Acc 51.1861%	 Train Loss 16.4936	


Val Acc 67.8571%	 Val Loss 2.6352



Epoch 2/30: 
Train Acc 70.4380%	 Train Loss 1.6450	


Val Acc 74.2857%	 Val Loss 0.7272



Epoch 3/30: 
Train Acc 82.2080%	 Train Loss 0.6002	


...

Val Acc 91.4286%	 Val Loss 0.9584



Epoch 29/30: 
Train Acc 98.9964%	 Train Loss 0.0514	


Val Acc 92.5000%	 Val Loss 1.0477



Epoch 30/30: 
Train Acc 98.9051%	 Train Loss 0.0655	


Val Acc 92.5000%	 Val Loss 0.9153


In [ ]:
torch.save(camera_model.state_dict(), "/content/drive/MyDrive/IDL Project/camera_model_final1.pt")

### Train with concatenated channel embeddings

In [ ]:

for epoch in range(0,10):

    train_acc, train_loss = train(model2, train_loader_camera,train_loader_sonar,optimizer, criterion,mode="both")
    
    print("\nEpoch {}/{}: \nTrain Acc {:.04f}%\t Train Loss {:.04f}\t".format(
        epoch + 1,
        20,
        train_acc,
        train_loss))
    
    val_acc, val_loss = validate(model2, val_loader_camera,val_loader_sonar, criterion,mode="both")
    print("Val Acc {:.04f}%\t Val Loss {:.04f}".format(val_acc, val_loss))




Epoch 1/20: 
Train Acc 19.8370%	 Train Loss 1.6785	


Val Acc 16.6667%	 Val Loss 1.6157



Epoch 2/20: 
Train Acc 19.3841%	 Train Loss 1.6733	


Val Acc 20.1389%	 Val Loss 1.6254



Epoch 3/20: 
Train Acc 19.2935%	 Train Loss 1.6723	


...

Val Acc 19.7917%	 Val Loss 1.6257



Epoch 9/20: 
Train Acc 18.7500%	 Train Loss 1.6741	


Val Acc 19.4444%	 Val Loss 1.6263



Epoch 10/20: 
Train Acc 21.6486%	 Train Loss 1.6600	


Val Acc 19.7917%	 Val Loss 1.6258


### AutoFusion

In [ ]:
class AutoFusion(nn.Module):

    def __init__(self, input_features, latent_features, num_classes=5):

        super().__init__()

        inp1 = input_features // 2

        # fuse in
        self.linear1 = nn.Linear(input_features, inp1)
        self.linear2 = nn.Linear(inp1, latent_features)

        # classification layer
        self.classification = nn.Linear(latent_features, num_classes)

        # fues_out
        self.linear3 = nn.Linear(latent_features, inp1)
        self.linear4 = nn.Linear(inp1, input_features)

        # weight tying
        self.linear3.weight = nn.Parameter(self.linear2.weight.T)
        self.linear4.weight = nn.Parameter(self.linear1.weight.T)

        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()

    def forward(self, concat_embd):

        # fuse in
        fused = self.tanh(self.linear1(concat_embd.to(device)))
        fused = self.relu(self.linear2(fused.to(device)))

        # classificatin probablities
        class_probs = self.classification(fused)

        # fuse_out
        reconst = self.tanh(self.linear3(fused.to(device)))
        reconst = self.linear4(reconst.to(device))

        return fused, reconst, class_probs

In [ ]:
def train(camera_dataloader, camera_model, sonar_dataloader, sonar_model, autofusion_model, criterion_mse, criterion_ce, optimizer):
  
    camera_model.eval()
    sonar_model.eval()
    autofusion_model.train()

    total_loss = 0.0
    num_correct=0
    for i, data in tqdm(enumerate(zip(camera_dataloader, sonar_dataloader))):

        # Move images to device
        camera_img = data[0][0].to(device)
        camera_label = data[0][1].to(device)
        sonar_img = data[1][0].to(device)
        sonar_label = data[1][1].to(device)

        # Get model outputs
        camera_embd = camera_model(camera_img)
        sonar_embd = sonar_model(sonar_img)

        #print(camera_embd)
        #print(sonar_embd)
        
        concat_embd = torch.concat((camera_embd, sonar_embd), dim=1)
        
        _, reconst_embd, class_probs = autofusion_model(concat_embd)

        class_loss = criterion_ce(class_probs, camera_label)
        reconstruction_loss_mse = criterion_mse(concat_embd, reconst_embd)
 
        num_correct += int((torch.argmax(class_probs, axis=1) == camera_label).sum())
        total_loss += float(reconstruction_loss_mse.item()+class_loss.item())

        optimizer.zero_grad()
        (reconstruction_loss_mse+class_loss).backward()
        optimizer.step()

    
    acc = 100 * num_correct / (8* len(camera_dataloader))
    total_loss = float(total_loss / len(sonar_dataloader))

    return total_loss,reconst_embd, acc

In [ ]:
def validate(camera_dataloader, camera_model, sonar_dataloader, sonar_model, autofusion_model, criterion_mse, criterion_ce, optimizer):
  
    autofusion_model.eval()
    camera_model.eval()
    sonar_model.eval()
    #batch_bar = tqdm(total=len(camera_dataloader), dynamic_ncols=True, position=0, leave=False, desc='Val', ncols=5)

    num_correct = 0.0
    total_loss = 0.0


    for i, data in tqdm(enumerate(zip(camera_dataloader, sonar_dataloader))):

        # Move images to device
        camera_img = data[0][0].to(device)
        camera_label = data[0][1].to(device)
        sonar_img = data[1][0].to(device)
        sonar_label = data[1][1].to(device)

        # Get model outputs
        with torch.inference_mode():
          camera_embd = camera_model(camera_img)
          sonar_embd = sonar_model(sonar_img)

        #print(camera_embd)
        #print(sonar_embd)
        
        concat_embd = torch.concat((camera_embd, sonar_embd), dim=1)
        
        with torch.inference_mode():
          _, reconst_embd, class_probs = autofusion_model(concat_embd)

        class_loss = criterion_ce(class_probs, camera_label)
        reconstruction_loss_mse = criterion_mse(concat_embd, reconst_embd)
 
        num_correct += int((torch.argmax(class_probs, axis=1) == camera_label).sum())
        total_loss += float(reconstruction_loss_mse.item()+class_loss.item())

        #optimizer.zero_grad()
        #(reconstruction_loss_mse+class_loss).backward()
        #optimizer.step()
        
    #batch_bar.close()
    acc = 100 * num_correct / (8* len(camera_dataloader))
    total_loss = float(total_loss / len(sonar_dataloader))
    return acc, total_loss

In [ ]:

camera_model_fuse = torch.nn.Sequential(*(list(camera_model.children())[:-1]))

sonar_model_fuse = torch.nn.Sequential(*(list(sonar_model.children())[:-1]))
autofusion_model = AutoFusion(1024, 256).to(device)

optimizer = torch.optim.AdamW(autofusion_model.parameters(), lr=1e-4, weight_decay=1e-2)

criterion_mse = torch.nn.MSELoss()
criterion_ce = torch.nn.CrossEntropyLoss()

In [ ]:
for epoch in range(30):

    train_loss,reconst_emb,train_acc = train(train_loader_camera, camera_model_fuse, train_loader_sonar, sonar_model_fuse, autofusion_model, criterion_mse, criterion_ce, optimizer)
    val_acc,val_loss=validate(val_loader_camera,camera_model_fuse,val_loader_sonar,sonar_model_fuse,autofusion_model,criterion_mse, criterion_ce, optimizer)
    print("Epoch: ",epoch)
    print("Training Loss is ",train_loss)
    print("Training Acc is ",train_acc)
    print("Validation Loss is ",val_loss)
    print("Validation Acc is ",val_acc)

    print("-"*10)

137it [00:51,  2.65it/s]
35it [00:12,  2.73it/s]

Epoch:  0
Training Loss is  132.89897157932992
Training Acc is  96.35036496350365
Validation Loss is  88.78703165075609
Validation Acc is  92.14285714285714
----------



137it [00:51,  2.66it/s]
35it [00:12,  2.78it/s]


Epoch:  1
Training Loss is  110.9551276710882
Training Acc is  99.72627737226277
Validation Loss is  78.03100041846878
Validation Acc is  92.5
----------


137it [00:52,  2.59it/s]
35it [00:12,  2.72it/s]

...

Epoch:  27
Training Loss is  44.35412993343349
Training Acc is  100.0
Validation Loss is  34.00653452356647
Validation Acc is  95.0
----------



137it [00:51,  2.67it/s]
35it [00:12,  2.72it/s]

Epoch:  28
Training Loss is  43.64853194778395
Training Acc is  99.81751824817518
Validation Loss is  33.58917569665812
Validation Acc is  95.0
----------



137it [00:51,  2.63it/s]
35it [00:16,  2.12it/s]

Epoch:  29
Training Loss is  43.07613009060191
Training Acc is  100.0
Validation Loss is  33.204454962496776
Validation Acc is  95.35714285714286
----------


In [ ]:
torch.save(autofusion_model.state_dict(), "/content/drive/MyDrive/IDL Project/auto_fusion_model.pt")

# GAN Fusion

In [ ]:
config = {
    'latent_dim': 512,
    'hidden_dim': 256,
    'lr': 1e-3
}

In [ ]:
class GanDiscriminator(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        """
        input_dim: dimension of the input latent codes
        hidden_dim: classifier's hidden dimension
        """
        super(GanDiscriminator, self).__init__()
        self._classifier = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),  nn.BatchNorm1d(hidden_dim),  nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, hidden_dim),  nn.BatchNorm1d(hidden_dim),  nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, 1)  
        )
        self._loss =  nn.BCEWithLogitsLoss()
        self._optimizer = torch.optim.Adam(self.parameters(), config['lr'])


    def optimize(self, clip):
        nn.utils.clip_grad_norm_(self.parameters(), clip)
        self._optimizer.step()


    def forward(self, input_latent, labels=None):
        output = self._classifier(input_latent)
        output_dict = {'output': output}
        if labels is not None:
            output_dict['loss'] = self._loss(output, labels)
            output_dict['accuracy'] = (torch.sigmoid(output).round() == labels).float().mean()
        return output_dict


class GanGenerator( nn.Module):
    def __init__(self, latent_dim, temperature=0.01):
        super(GanGenerator, self).__init__()
        self._latent_mapper =  nn.Sequential(
             nn.Linear(latent_dim, 2*latent_dim),  nn.BatchNorm1d(2*latent_dim),  nn.LeakyReLU(0.2),
             nn.Linear(2*latent_dim, 2*latent_dim),  nn.BatchNorm1d(2*latent_dim),  nn.LeakyReLU(0.2),
             nn.Linear(2*latent_dim, latent_dim)
        )

        self._temperature = temperature
        self._ce_loss =  nn.BCEWithLogitsLoss()
        self._optimizer =  torch.optim.Adam(self.parameters(), config['lr'])

    def add_noise(self, latent):
        """
        Adds standard normal noise to a latent vector
        """
        return self._temperature*torch.randn(latent.shape).to(device) + latent

    def optimize(self, clip):
        nn.utils.clip_grad_norm_(self.parameters(), clip)
        self._optimizer.step()

    def forward(self, target_latent, discriminator):
        # add noise to input data
        target_latent = self.add_noise(target_latent)
        z_g = self._latent_mapper(target_latent)
        output_dict = {}
        # continue discriminator part
        
        if discriminator is not None:
            predicted = discriminator(z_g)['output']
            # the generator wants the disc to think these as true
            desired = torch.ones_like(predicted[:,0].reshape(-1,1))
            # print(predicted.shape, desired.shape)
            output_dict['loss'] = self._ce_loss(predicted, desired)
            output_dict['accuracy'] = (torch.sigmoid(predicted).round() == desired).float().mean()

        output_dict.update({
            'z_g': z_g
            })
        return output_dict

In [ ]:
class GanFusionSingle(nn.Module):
    """docstring for GanFusionSingle: fusion module for one target modality"""
    def __init__(self,
                 config,
                 generator,
                 discriminator):
        """
        target_latent: target modality's latent code
        compl_latent: complementary modalities' latent codes (concatenated)
        """
        super(GanFusionSingle, self).__init__()
        self.config = config
        self.generator = generator
        self.discriminator = discriminator
        self.gen_metrics = None
        self.disc_metrics = None

    def forward(self, z_dict, mode, stage=None):
        # refer to the paper for conventions
        if mode != 'train':
            stage = 'gen'

        if stage == 'disc_real':
            z_tr = z_dict['compl']
            labels = torch.ones_like(z_tr[:,0].reshape(-1,1))
            output = self.discriminator(z_tr, labels)
            # add autofusion's loss as well

            output['z_g'] = self.generator(z_dict['target'], self.discriminator)['z_g']
            self.disc_metrics = {
                'drl': output['loss'],
                'dracc': output['accuracy']
            }
        elif stage == 'disc_fake':
            # get the generated latent z_g
            z_g = self.generator(z_dict['target'], self.discriminator)['z_g']
            labels = torch.zeros_like(z_g[:,0].reshape(-1,1))
            output = self.discriminator(z_g, labels)
            output['z_g'] = z_g
            self.disc_metrics = {
                'dfl': output['loss'],
                'dfacc': output['accuracy']
            }
        elif stage == 'gen':
            output = self.generator(z_dict['target'], self.discriminator)
            self.gen_metrics = {
                'gl': output['loss'],
                'gacc': output['accuracy']
            }
        else:
            raise ValueError(f'invalid stage: {stage}')
        return output


class GanFusion(nn.Module):
    """docstring for GanFusion: contains GAN-Fusion module for all the
    modalities"""
    def __init__(self, config):
        super(GanFusion, self).__init__()
        self.sonar_gan = GanFusionSingle(config,
                                        GanGenerator(config['latent_dim']),
                                        GanDiscriminator(config['latent_dim'],
                                                        config['hidden_dim']))
        self.camera_gan = GanFusionSingle(config,
                                          GanGenerator(config['latent_dim']),
                                          GanDiscriminator(config['latent_dim'],
                                                        config['hidden_dim']))

        self.ff_input_features = config['latent_dim']*2
        self.feed_forward = nn.Sequential(
            nn.Linear(self.ff_input_features, self.ff_input_features//2),
            nn.Tanh(),
            nn.Linear(self.ff_input_features//2, config['latent_dim']),
            nn.ReLU()
            )

    def z_fuse(self, fusion_dict):
        z_fuse_t = fusion_dict['sonar_dict']['z_g']
        z_fuse_s = fusion_dict['camera_dict']['z_g']
        return self.feed_forward(torch.cat((z_fuse_t,
                                            z_fuse_s), dim=-1))

    def get_loss(self, fusion_dict):
        return fusion_dict['sonar_dict']['loss'] + \
               fusion_dict['camera_dict']['loss']

    def forward(self, latent_dict, mode, stage=None):
        z_t = latent_dict['z_sonar']
        z_s = latent_dict['z_camera']

        fusion_dict = {
            'sonar_dict': self.sonar_gan({'target': z_t, 'compl': z_s}, mode, stage),
            'camera_dict': self.camera_gan({'target': z_s, 'compl': z_t}, mode, stage)}

        return {
            'z': self.z_fuse(fusion_dict),
            'loss': self.get_loss(fusion_dict)
        }

In [ ]:
class FinalModel(nn.Module):
    """docstring for GanFusionSingle: fusion module for one target modality"""
    def __init__(self, config, camera_model, sonar_model, num_classes):
        super(FinalModel, self).__init__()
        self.camera_model = camera_model
        self.camera_model.eval()
        self.sonar_model = sonar_model
        self.sonar_model.eval()
        self.gan_fusion = GanFusion(config)
        self.class_layer = nn.Linear(config['latent_dim'], num_classes)

    def forward(self,input_sonar, input_camera, mode, stage):
        sonar_embedding = self.sonar_model(input_sonar)
        camera_embedding = self.camera_model(input_camera)
        latent_dict = {
            'z_sonar': sonar_embedding,
            'z_camera': camera_embedding
        }

        gan_fusion_output = self.gan_fusion(latent_dict, mode, stage)
        class_output = self.class_layer(gan_fusion_output['z'])
        return gan_fusion_output, class_output

In [ ]:
final_model = FinalModel(config, camera_model_fuse, sonar_model_fuse, 5).to(device)

In [ ]:
optimizer_gan =  torch.optim.Adam(final_model.parameters(), 1e-5)
criterion_gan = nn.CrossEntropyLoss()

In [ ]:
def train_GAN(model, camera_dataloader,sonar_dataloader, optimizer_gan):
    model.gan_fusion.train()
    # Progress Bar 
    batch_bar = tqdm(total=len(camera_dataloader), dynamic_ncols=True, leave=False, position=0, desc='Train', ncols=5) 
    
    num_correct = 0
    total_loss = 0
    dataloader=zip(camera_dataloader,sonar_dataloader)
    losses = {'disc_real':0, 'disc_fake':0, 'gen':0}

    for i, data in enumerate(dataloader):
        
        image_camera=data[0][0]
        labels=data[0][1] #Same labels for both 
        image_sonar=data[1][0]
        # images=torch.concat([image_camera,image_sonar],dim=1)
        
        

        image_camera, image_sonar, labels = image_camera.to(device), image_sonar.to(device), labels.to(device)
        
        # for stage in ['disc_fake']:

        
        for stage in ['disc_real', 'disc_fake', 'gen']:
            optimizer_gan.zero_grad() # Zero gradients
            output_dict, class_output = model(image_sonar, image_camera, 'train', stage)
            loss_class = criterion_gan(class_output, labels)
            if(stage!='gen'):
                loss = output_dict['loss']
            else:
                loss= output_dict['loss'] + loss_class
            loss.backward()
            optimizer_gan.step()
            losses[stage] += output_dict['loss'].item()

            # num_correct += int((torch.argmax(outputs, axis=1) == labels).sum())
        
        total_loss += float(loss_class.item())
        
        num_correct += int((torch.argmax(class_output, axis=1) == labels).sum())
        # total_loss += float(loss.item())

        batch_bar.set_postfix(
            acc="{:.04f}%".format(100 * num_correct / (8*(i + 1))),
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.04f}".format(float(optimizer_gan.param_groups[0]['lr'])),
            disc_real="{:.04f}".format(float(losses['disc_real'] / (i + 1))),
            disc_fake="{:.04f}".format(float(losses['disc_fake'] / (i + 1))),
            gen="{:.04f}".format(float(losses['gen'] / (i + 1))))

            
        
        
        batch_bar.update()

    gan_training_metrics['train_accuracy'].append(100 * num_correct / (8*(i + 1)))
    gan_training_metrics['train_class_loss'].append(float(total_loss / (i + 1)))
    gan_training_metrics['train_disc_real'].append(float(losses['disc_real'] / (i + 1)))
    gan_training_metrics['train_disc_fake'].append(float(losses['disc_fake'] / (i + 1)))
    gan_training_metrics['train_gen'].append(float(losses['gen'] / (i + 1)))
    
    batch_bar.close()
    print("Training : ")
    print("Accuracy:","{:.04f}%".format(100 * num_correct / (8*(i + 1))))
    print("Classification Loss","{:.04f}".format(float(total_loss / (i + 1))))
    print("Discriminator Real","{:.04f}".format(float(losses['disc_real'] / (i + 1))))
    print("Discriminator Fake","{:.04f}".format(float(losses['disc_fake'] / (i + 1))))
    print("Generator","{:.04f}".format(float(losses['gen'] / (i + 1))))

    total_loss = float(total_loss / len(camera_dataloader))

    return total_loss

In [ ]:
def valid_GAN(model, camera_dataloader,sonar_dataloader):
    model.eval()
    # Progress Bar 
    batch_bar = tqdm(total=len(camera_dataloader), dynamic_ncols=True, leave=False, position=0, desc='Valid', ncols=5) 
    
    num_correct = 0
    total_loss = 0
    dataloader=zip(camera_dataloader,sonar_dataloader)
    losses = {'disc_real':0, 'disc_fake':0, 'gen':0}

    for i, data in enumerate(dataloader):
        
        image_camera=data[0][0]
        labels=data[0][1] #Same labels for both 
        image_sonar=data[1][0]
        # images=torch.concat([image_camera,image_sonar],dim=1)
        
        

        image_camera, image_sonar, labels = image_camera.to(device), image_sonar.to(device), labels.to(device)
        
        # for stage in ['disc_fake']:

        with torch.no_grad():
        
            for stage in ['disc_real', 'disc_fake', 'gen']:
                output_dict, class_output = model(image_sonar, image_camera, 'train', stage)
                loss_class = criterion_gan(class_output, labels)
                if(stage!='gen'):
                    loss = output_dict['loss']
                else:
                    loss= output_dict['loss'] + loss_class
                losses[stage] += output_dict['loss'].item()
            
            total_loss += float(loss_class.item())
        
        num_correct += int((torch.argmax(class_output, axis=1) == labels).sum())

        batch_bar.set_postfix(
            acc="{:.04f}%".format(100 * num_correct / (8*(i + 1))),
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            disc_real="{:.04f}".format(float(losses['disc_real'] / (i + 1))),
            disc_fake="{:.04f}".format(float(losses['disc_fake'] / (i + 1))),
            gen="{:.04f}".format(float(losses['gen'] / (i + 1))))
            
        
        
        batch_bar.update()
    gan_training_metrics['val_accuracy'].append(100 * num_correct / (8*(i + 1)))
    gan_training_metrics['val_class_loss'].append(float(total_loss / (i + 1)))
    gan_training_metrics['val_disc_real'].append(float(losses['disc_real'] / (i + 1)))
    gan_training_metrics['val_disc_fake'].append(float(losses['disc_fake'] / (i + 1)))
    gan_training_metrics['val_gen'].append(float(losses['gen'] / (i + 1)))

    batch_bar.close()
    print("Validation: ")
    print("Accuracy:","{:.04f}%".format(100 * num_correct / (8*(i + 1))))
    print("Classification Loss","{:.04f}".format(float(total_loss / (i + 1))))
    print("Discriminator Real","{:.04f}".format(float(losses['disc_real'] / (i + 1))))
    print("Discriminator Fake","{:.04f}".format(float(losses['disc_fake'] / (i + 1))))
    print("Generator","{:.04f}".format(float(losses['gen'] / (i + 1))))

    total_loss = float(total_loss / len(camera_dataloader))

    return total_loss

In [ ]:
for i in range(30):
    print('Epoch',i+1,'/30')
    train_GAN(final_model, train_loader_camera, train_loader_sonar, optimizer_gan)
    valid_GAN(final_model, val_loader_camera, val_loader_sonar)

Epoch 1 /30


Training : 
Accuracy: 81.4781%
Classification Loss 1.1550
Discriminator Real 1.4558
Discriminator Fake 1.3550
Generator 1.4616


Validation: 
Accuracy: 90.0000%
Classification Loss 0.7392
Discriminator Real 1.2574
Discriminator Fake 1.3342
Generator 1.4407
Epoch 2 /30


Training : 
Accuracy: 92.2445%
Classification Loss 0.4718
Discriminator Real 1.3088
Discriminator Fake 1.4653
Generator 1.3329


Validation: 
Accuracy: 91.0714%
Classification Loss 0.3285
Discriminator Real 1.0621
Discriminator Fake 1.4404
Generator 1.3345
Epoch 3 /30


...

Validation: 
Accuracy: 94.6429%
Classification Loss 0.1474
Discriminator Real 0.5331
Discriminator Fake 2.1202
Generator 0.8574
Epoch 29 /30


Training : 
Accuracy: 100.0000%
Classification Loss 0.0006
Discriminator Real 0.5960
Discriminator Fake 1.9367
Generator 0.9659


Validation: 
Accuracy: 95.0000%
Classification Loss 0.1439
Discriminator Real 0.5294
Discriminator Fake 2.1217
Generator 0.8569
Epoch 30 /30


Training : 
Accuracy: 100.0000%
Classification Loss 0.0006
Discriminator Real 0.5896
Discriminator Fake 1.9364
Generator 0.9664


Validation: 
Accuracy: 93.9286%
Classification Loss 0.2060
Discriminator Real 0.5293
Discriminator Fake 2.1319
Generator 0.8513
